<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/QuantumApplicationAlgorithm_QiskitNature0_7_2_GettingStarted_mkbahk_20251208.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python 3.9 ~ 3.11 환경에서 아래 순서대로 설치
!pip install qiskit[visualization]==1.2.4
!pip install qiskit-aer==0.15.1
!pip install qiskit-algorithms==0.3.1
!pip install qiskit-nature[pyscf]==0.7.2
!pip install pyscf==2.6.0
!pip install py3Dmol

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 10.3 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=0d9da97ebc2ef0f4383d3ab6c89c8d3c664d63def98e98c8d4db7f1873c5f7f1
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

In [4]:
# Use PySCF, a classical computational chemistry software
# package, to compute the one-body and two-body integrals in
# electronic-orbital basis, necessary to form the Fermionic operator
driver = PySCFDriver(
    atom='H .0 .0 .0; H .0 .0 0.735',
    #atom="O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0",
    unit=DistanceUnit.ANGSTROM,
    basis='sto3g',
)

#ElectronicStructureProblem
problem = driver.run()

print(f"Number of Particles(전자수|입자수): {problem.num_particles}")
print(f"Number of Spatial Orbitals(공간 괘도 수): {problem.num_spatial_orbitals}")
#각 공간 오비탈에 업/다운 스핀이 있으므로 보통 공간 오비탈 수의 2배 → 스핀 오비탈 수
#(스핀 궤도 수라고도 불림)
print(f"Number of Spin Orbitals(스핀 오비탈 수): {problem.num_spin_orbitals}")


Number of Particles(전자수|입자수): (1, 1)
Number of Spatial Orbitals(공간 괘도 수): 2
Number of Spin Orbitals(스핀 오비탈 수): 4


In [5]:
# setup the qubit mapper
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper(num_particles=problem.num_particles)

In [6]:
# setup the classical optimizer for the VQE
from qiskit_algorithms.optimizers import L_BFGS_B

optimizer = L_BFGS_B()

In [7]:
# setup the estimator primitive for the VQE
from qiskit.primitives import Estimator

estimator = Estimator()

In [8]:
# setup the ansatz for VQE
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

In [9]:
#ansatz.decompose().draw('mpl')
ansatz.decompose().draw()

┌───┐┌────────────────────┐┌────────────────────┐»
q_0: ┤ X ├┤0                   ├┤0                   ├»
     └───┘│  exp(-it IY)(t[0]) ││  exp(-it YI)(t[1]) │»
q_1: ─────┤1                   ├┤1                   ├»
          └────────────────────┘└────────────────────┘»
«     ┌───────────────────────────┐
«q_0: ┤0                          ├
«     │  exp(-it (XY + YX))(t[2]) │
«q_1: ┤1                          ├
«     └───────────────────────────┘

In [10]:
# set up our actual VQE instance
from qiskit_algorithms import VQE

vqe = VQE(estimator, ansatz, optimizer)
# ensure that the optimizer starts in the all-zero state which corresponds to
# the Hartree-Fock starting point
vqe.initial_point = [0] * ansatz.num_parameters

In [11]:
# prepare the ground-state solver and run it
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

algorithm = GroundStateEigensolver(mapper, vqe)

electronic_structure_result = algorithm.solve(problem)
electronic_structure_result.formatting_precision = 6

아래 출력은 VQE가 찾은 바닥상태가 전자 수, 총 스핀, 스핀 방향까지 모두 물리적으로 올바른 상태임을 검증한 진단 결과입니다.

In [12]:
print(electronic_structure_result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275
  - computed part:      -1.857275
~ Nuclear repulsion energy (Hartree): 0.719969
> Total ground state energy (Hartree): -1.137306
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.388949]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388949]
    - computed part:      [0.0  0.0  1.388949]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


1. Electronic ground state energy(전자 바닥상태 에너지): 전자 운동 에너지, 전자–핵 쿨롱 인력, 전자–전자 쿨롱 반발<BR> Computed Part: 드라이버가 계산한 전자 에너지의 실제 수치, 일부 패키지에서는 frozen-core보정, scalar relativistic correction등이 분리되기도 하지만, 👉 이 경우는 전자 에너지 전부를 직접 계산했다는 뜻입니다.

2. Nuclear repulsion energy(핵반발에너지): 핵들 사이의 정전기적(쿨롱) 반발 에너지, 핵은 양전하
전자는 고려하지 않음, 고정된 핵 위치에서의 순수 고전적 에너지
3. Total ground state energy(분자의 전체 바닥상태 에너지) = Electronic ground state energy + Nuclear repulsion energy, 실험적 분자 예너지, 결합에너지, 반응에어지 계산에 사용

4. DIPOLE MOMENTS(쌍극자 모멘트)
5. Nuclear dipole moment(행 쌍극자 모멘트)
6. Electronic dipole moment(전자 썅극자 모멘트)
7. Dipole moment(썅극자 모멘트)